# Creating an AI newsletter assistant with Composio and Groq

This notebook demonstrates how to easily connect AI agents to external applications like Gmail, Slack, GitHub and many more using Composio. More specifically, we'll be setting up an assistant that will automatically look for email newsletters in our inbox, summarize and enrich their content with relevant links and contextually relevant information, and then send these summaries to us in a well-formatted and easy to digest email.

### About Composio

Composio is a toolset designed to streamline and optimize the connection and interaction between AI Agents/LLMs and various APIs/services. Composio supports integrations with a variety of popular apps including Google Apps, Github, Slack, and also system tools like Code Interpreter, File Manager and Databases, in just a few lines of code. It also provides pre-defined actions(functions), allowing agents to interact with these integrations.  

Let's get started!

### Setup

In [ ]:
# Import packages
import dotenv
from composio_langchain import App, ComposioToolSet
from langchain_groq import ChatGroq
from datetime import datetime

# Load environment variables from .env file
dotenv.load_dotenv()

We are going to be using Composio's Gmail and Tavily tools in this guide. To use these tools, we first need to create integrations with these apps. This can be done using the command line. You'll need a Tavily API key, which you can get [here](https://app.tavily.com/)  - 

To create an integration with Gmail, run the command below - 

In [ ]:
composio add gmail

Similarly, to create an integration with Tavily -

In [ ]:
composio add tavily

Make sure you have a Groq API key and set it as an environment variable. You can get your key [here](https://console.groq.com/) for free.

In [ ]:
# Initialize the model
llm = ChatGroq(model="llama3.1-70b-versatile", stop_sequences=["\n\n"])

### Initialize Toolset and get tools

Now we initialize Composio's toolset which provides all the necessary tools and actions we need. For this guide, we will get the Gmail and Tavily tools from the toolset. The Gmail tools will allow the agent to perform actions like fetching and sending emails. The tavily tool allows the agent to search the web and gather more information about the topics of the newsletter.

In [ ]:
# Initialize the ComposioToolSet
toolset = ComposioToolSet()

# Get the tools from the ComposioToolSet
gmail_tools = toolset.get_tools(apps=[App.GMAIL])
tavily_search = toolset.get_tool(apps=[App.TAVILY])


### Define Agents

Next, we define our agents. We'll create three agents, where each agent performs a specific task. 

- <b>email_fetcher_agent</b> - Fetches newsletter emails from Gmail inbox using `gmail_tools`.
- <b>summarizer_agent</b> - Summarizes the fetched emails, adds relevant information using the `tavily_search` tool.
- <b>email_sender_agent</b> - Sends an email with the summarized content to the specified email address using `gmail_tools`.

In [ ]:
from crewai import Agent, Crew, Process, Task

# Define the Email Fetcher Agent
email_fetcher_agent = Agent(
    role="Email Fetcher Agent",
    goal="Fetch recent newsletter emails from the inbox. Please look for emails with 'newsletter' or 'digest' only for last 7 days. Don't add any other unncessary filters.",
    verbose=True,
    memory=True,
    backstory=f"You are an expert in retrieving and organizing email content, with a keen eye for identifying relevant newsletters. Today's date is {datetime.now().strftime('%B %d, %Y')}. You are writing an email to a reader who is interested in the stock market and trading.",
    llm=llm,
    allow_delegation=False,
    tools=gmail_tools,
)

# Define the Summarizer Agent
summarizer_agent = Agent(
    role="Summarizer Agent",
    goal="Summarize the content of newsletter emails, highlighting key information and trends. ",
    verbose=True,
    memory=True,
    backstory=f"You are an expert in analyzing and summarizing complex information, with a talent for distilling essential points from various sources. Today's date is {datetime.now().strftime('%B %d, %Y')}. You are writing an email to a reader who is interested in the stock market and trading.",
    llm=llm,
    allow_delegation=False,
    tools=tavily_search,
)

# Define the Email Sender Agent
email_sender_agent = Agent(
    role="Email Sender Agent",
    goal="Send the summarized newsletter content via email to investtradegame@gmail.com with a professional and engaging format",
    verbose=True,
    memory=True,
    backstory=f"You are an expert in composing and sending emails with well-formatted, visually appealing content. You have a knack for creating engaging subject lines and structuring information for easy readability. Today's date is {datetime.now().strftime('%B %d, %Y')}. You are writing an email to a reader who is interested in the stock market and trading.",
    llm=llm,
    allow_delegation=False,
    tools=gmail_tools,
)

### Define Tasks

Now, we define the tasks that the agents need to perform. These tasks tell the agents the actions they need to perform to achieve a particular goal. Here, we define several tasks that our agents will execute to summarize our newsletters - 

- <b>fetch_emails_task</b> - This task provides instructions on fetching relevant newsletter emails from the inbox.
- <b>summarize_emails_task</b> - This task summarizes the fetched newsletter emails. It uses the `tavily_search` tool to add more information to the summary if necessary.
- <b>send_summary_task</b> - This task details how the summaries are formatted and sent to the user.

In [ ]:
# Define the task for fetching emails
fetch_emails_task = Task(
    description=(
        "Fetch the most recent newsletter emails from the inbox. "
        "Look for emails with subjects containing words like 'newsletter', 'update', or 'digest'. "
        "Retrieve the content of these emails, including any important links or attachments. "
        "Pay special attention to newsletters from reputable sources and industry leaders."
    ),
    expected_output="A detailed list of recent newsletter emails with their content, including any relevant links or attachments",
    tools=gmail_tools,
    agent=email_fetcher_agent,
)

# Define the task for summarizing emails
summarize_emails_task = Task(
    description=(
        "Summarize the content of the fetched newsletter emails. "
        "Create a concise yet comprehensive summary highlighting the key points from each newsletter. "
        "Organize the summaries in a clear and readable format, grouping related topics if applicable. "
        "Include any important links, statistics, or data points that add value to the summary. "
        "Identify and highlight any emerging trends or significant developments across the newsletters."
        "Use the tavily_search tool to add to the summary. Use the tavily_search tool to search for relevant keywords in the summary and include the information from the search results if the information is not found in the summary, formatted in a clear and concise manner. "
    ),
    expected_output="A comprehensive and well-structured summary of the newsletter emails, including key points, trends, and important links",
    tools=tavily_search,
		agent=summarizer_agent,
    context=[fetch_emails_task],
)

# Define the task for sending the summary email
send_summary_task = Task(
    description=(
        "Compose and send an email containing the summarized newsletter content. "
        "Use the Gmail API to send the email to investtradegame@gmail.com. "
        "Ensure the email has a clear, engaging subject line and well-formatted content. "
        "Use the following structure for the email:\n\n"
        f"Subject: Your Weekly News Digest - {datetime.now().strftime('%B %d, %Y')}\n\n"
        "<h1>Weekly News Digest</h1>\n\n"
        "<p>Dear Reader,</p>\n\n"
        "<p>Here's your curated summary of this week's top news items and insights:</p>\n\n"
        "[Insert summarized content here]\n\n"
        "Each main section should be separated by a horizontal rule, like this:\n"
        "<hr>\n\n"
        "Structure the content logically, with clear sections for each summarized newsletter or topic area.\n"
        "Use appropriate HTML formatting such as <strong> for headlines, "
        "<ul> and <li> for bullet points, and <br> for line breaks to enhance readability.\n\n"
        "Include relevant links using HTML anchor tags: <a href='URL'>Link Text</a>\n\n"
        "Include a brief introduction at the beginning to set the context and a conclusion at the end "
        "to summarize the key takeaways and trends observed across the newsletters.\n\n"
        "<footer>\n"
        "<p>For more details on these stories, click on the provided links or stay tuned for our next update. "
        "If you have any questions or feedback, please don't hesitate to reach out.</p>\n\n"
        "<p>Best regards,<br>Your Newsletter Summary Team</p>\n"
        "</footer>\n\n"
        "Important: Ensure all HTML tags are properly closed and nested correctly."
    ),
    expected_output="Confirmation that the well-formatted, detailed summary email with important links has been sent to investtradegame@gmail.com",
    tools=gmail_tools,
    agent=email_sender_agent,
    context=[summarize_emails_task],
)

### Run the agents

Finally, we define a crew with all of these agents and tasks, and then kick of the crew to get the results - 

In [ ]:
# Define the crew with the agents and tasks
crew = Crew(
    agents=[email_fetcher_agent, summarizer_agent, email_sender_agent],
    tasks=[fetch_emails_task, summarize_emails_task, send_summary_task],
    process=Process.sequential,
)

# Kickoff the process and print the result
result = crew.kickoff()
print("Newsletter Summary Process Completed:")
print(result)

### Conclusion

In this guide, we successfully used Groq to connect to the LLM and Composio allowed us to easily connect to Gmail and Tavily and use their tools to create an AI agent that can fetch newsletter emails, summarize them and then send the summary as a mail to the user.

### Connect with us

Join the [Composio Discord](https://discord.com/invite/cNruWaAhQk) and [Groq Discord](https://discord.com/invite/groq) servers to discuss your ideas and get support from the community.

### Learn More

Composio: https://composio.dev/
Composio Docs: https://docs.composio.dev/introduction/intro/overview